# Tarea 4: implementar un modelo de inferencia asíncrona

## Tarea 4.1: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from botocore.exceptions import ClientError

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

Guarde el modelo del laboratorio de entrenamiento y ajuste en el bucket predeterminado de Amazon Simple Storage Service (Amazon S3). Configure un modelo usando **create_model** y configure **ModelDataUrl** para referenciar al modelo entrenado.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

Cargue los registros asíncronos en el bucket predeterminado de Amazon S3.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="asynchronous_records.csv", Bucket=bucket, Key=f"{prefix}/asynchronous_records.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
input_location = f"s3://{bucket}/{prefix}/asynchronous_records.csv"

## Tarea 4.2: crear un punto de enlace a partir del modelo reentrenado y sintetizado que se proporciona

Amazon SageMaker Asynchronous Inference es una capacidad de SageMaker que pone las solicitudes entrantes en una cola y las procesa de forma asíncrona. Esta opción es ideal para solicitudes con grandes tamaños de carga (hasta 1 GB), tiempos de procesamiento extensos (hasta 15 minutos) y requisitos de latencia casi en tiempo real. Con Asynchronous Inference, puede reducir los costos mediante el escalado automático del conteo de instancias a cero cuando no haya solicitudes para procesar. Por lo tanto, solo paga cuando el punto de enlace procesa solicitudes.

Puede crear un punto de enlace asíncrono con el SDK para Python de SageMaker en tres pasos. Los pasos son los mismos que se usan para los puntos de enlace sin servidores y en tiempo real, pero tienen distintas configuraciones:
1. Cree un modelo en SageMaker.
2. Cree una configuración para un punto de enlace HTTPS.
3. Cree un punto de enlace HTTPS.

Ya creó un modelo. Ahora está listo para crear un punto de enlace y su configuración. 

Primero, establezca la configuración del punto de enlace y el tipo de instancia que desea usar. Luego, llame a la API CreateEndpointConfig.

Para crear una configuración del punto de enlace, debe establecer las siguientes opciones:
- **VariantName**: el nombre de la variante de producción (uno o más modelos en producción)
- **ModelName**: el nombre del modelo que desea alojar. Este es el nombre que especificó cuando creó el modelo.
**InstanceType**: es el tipo de instancia de cómputo.
- **S3OutputPath**: la ubicación donde se cargarán las salidas de respuesta cuando no se proporcione una ubicación en la solicitud
- **MaxConcurrentInvocationsPerInstance**: (opcional) la cantidad máxima de solicitudes simultáneas enviadas por el cliente de SageMaker al contenedor del modelo.

De forma opcional, también puede configurar NotificationConfig a fin de seleccionar un tema de Amazon Simple Notification Service (Amazon SNS) que publique notificaciones cuando una solicitud de inferencia se realice correctamente o falle. En este laboratorio, no necesita configurar esta opción.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-asynchronous-endpoint-{}'.format(create_date)                              
output_location = f"s3://{bucket}/{prefix}/output"

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant.
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
            
        } 
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": output_location
        },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker chooses an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

A continuación, cree un punto de enlace. Cuando crea un punto de enlace asíncrono, SageMaker inicia las instancias de cómputo de machine learning (ML) e implementa el modelo según lo especificado en la configuración. Consulte [Inferencias asíncronas](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html) para obtener más información sobre las opciones disponibles para usted con los puntos de enlace asíncronos.

Cuando el punto de enlace está en funcionamiento, la función auxiliar imprime el Amazon Resource Name (ARN) del punto de enlace. La creación del punto de enlace puede demorar hasta 7 minutos en ejecutarse.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


En SageMaker Studio, puede ver los detalles del punto de enlace debajo de la pestaña **Endpoints** (Puntos de enlace).

El próximo paso abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **asynchronous_inference.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **asynchronous_inference.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones visibles mientras explora el punto de enlace.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar el punto de enlace, seleccione la pestaña **asynchronous_inference.ipynb** para volver al cuaderno.

1. Seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
2. Seleccione **Deployments** (Implementaciones).
3. Seleccione **Endpoints** (Puntos de enlace).

Verá la pestaña **Endpoints** (Puntos de enlace) en SageMaker Studio.

4. Seleccione el punto de enlace que tenga **income-model-asynchronous-** en la columna **Name** (Nombre).

Si el punto de enlace no aparece, seleccione el ícono de actualización hasta que aparezca en la lista.

Verá la pestaña **ENDPOINT DETAILS** (DETALLES DEL PUNTO DE ENLACE) en SageMaker Studio.

5. Vaya a la pestaña **AWS settings** (Configuración de AWS).

Si abrió el punto de enlace antes de que terminara de crearse, seleccione el ícono de actualización hasta que el valor de **Endpoint status** (Estado del punto de enlace) cambie de *Creating* (Creando) a *InService* (En funcionamiento).

En **Endpoint type** (Tipo de punto de enlace), verá **Async** (Asínc.). En la sección **Endpoint runtime settings** (Configuración del tiempo de ejecución del punto de enlace), verá las configuraciones que eligió antes en el cuaderno.

## Tarea 4.3: invocar un punto de enlace para una inferencia asíncrona con registros de clientes asíncronos

Después de que implemente su modelo con los servicios de alojamiento de SageMaker, puede enviarle datos de prueba al modelo en ese punto de enlace para probarlo.

Para probar un punto de enlace asíncrono, debe incluir la ubicación de la entrada de Amazon S3 en la llamada API. Para este laboratorio, puede probar el punto de enlace con un archivo asynchronous_records.csv que hay en el bucket S3 predeterminado de SageMaker con 100 registros de clientes. Si la acción se realiza correctamente, el servicio envía una respuesta HTTP 202.

In [ ]:
#send-test-file
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)

print(response)

output_key = response['OutputLocation'].split("/", 3)[3]
print('\nThe output key is: {}'.format(output_key))

Revise la ubicación de la salida para ver si se procesó la inferencia. Cuando se haya procesado, imprima las puntuaciones de la predicción para todos los clientes incluidos en la invocación.

In [ ]:
#get-output
def get_output():
    while True:
        try:
            return sagemaker.session.Session().read_s3_file(bucket=bucket, key_prefix=output_key)
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("Waiting for output...")
                time.sleep(2)
                continue
            raise

output = get_output()
print(f"Predictions for the 100 customers: {output}")

## Tarea 4.4: eliminar el punto de enlace

La eliminación de un punto de enlace se puede lograr en tres pasos. Primero, elimine el punto de enlace. Luego, elimine la configuración del punto de enlace. Por último, si ya no necesita el modelo que implementó, elimínelo.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### Conclusión

¡Felicitaciones! Usó SageMaker para crear correctamente un punto de enlace asíncrono, con el SDK para Python de SageMaker, y para invocarlo.

La próxima tarea del laboratorio se centra en la transformación del lote.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Tarea 5: utilizar la transformación por lotes para obtener inferencias a partir de un gran conjunto de datos**.